# Рынок заведений общественного питания Москвы

Мы решили открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Вместе с партнёрами мы решились обратиться к инвесторам. Их интересует текущее положение дел на рынке — сможем ли мы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов?

**Цель исследования** - определить наиболее перспективные характеристики будущего заведения: вид, количество посадочных мест, а также район расположения.

Исследование мы проведем на основе открытых данных.  

**Ход исследования**

Для анализа мы используем следующие данные:
* характеристику заведений общественного питания Москвы;
* перечень улиц города Москвы с принадлежностью к району и административному округу. 

О качестве данных нам ничего неизвестно, поэтому перед проведением исследования понадобится обзор данных.
Мы проведем предобработку данных: обработаем пропуски и дубликаты в случае наличия, приведем данные к необходимым типам, проведем дополнительные расчеты.

Затем мы проведем исследовательский анализ - оценим распространенность различных видов заведений, влияние сетевого характера, расположения и числа посадочных мест на количество заведений, дополнительно проанализируем улицы, на которых расположены заведения.

На основе анализа подготовим презентацию для инвесторов.

Таким образом, решение пройдет в три этапа.

## Обзор данных и предобработка

### Обзор данных

In [1]:
pip install nbconvert[webpdf]

Note: you may need to restart the kernel to use updated packages.


Составим представление о данных. Импортируем специальную библиотеку pandas для обработки и анализа данных, а также другие библиотеки. 

In [2]:
import pandas as pd # импорт библиотеки pandas
import seaborn as sns # импорт библиотеки seaborn для работы с графиками

import matplotlib.pyplot as plt # импорт части библиотеки matplotlib
import plotly.express as px
import plotly.graph_objects as go
import re  # импорт библиотеки re для работы с регулярными выражениями

import numpy as np # импорт библиотеки numpy для математических вычислений


Обновим библиотеку matplotlib, чтобы нам были доступны все функции при построении графиков.

In [3]:
pip install --upgrade matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 17.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 KB 14.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
# чтение файла с данными и сохранение в rest_data
try:
    # адрес в локальном окружении
    rest_data = pd.read_csv('rest_data.csv')
except:
    # адрес в среде 'practicum'
    rest_data = pd.read_csv('/datasets/rest_data.csv')

Выведем на экран первые пять строк таблицы

In [5]:
# Запрос демонстрации первых 5 строк таблицы
display(rest_data.head(5)) # получение первых 5 строк таблицы

,id,object_name,chain,object_type,address,number
0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50


Прочитаем таблицы с данными о принадлежности улицы к району. 

In [6]:
try:
    streets = pd.read_csv('streets.csv')
except:

    url = 'https://frs.noosphere.ru/xmlui/bitstream/handle/20.500.11925/714058/mosgaz-streets.csv?sequence=1&isAllowed=y'
    streets = pd.read_csv(url)

In [7]:
streets.head(5)

,streetname,areaid,okrug,area
0,Выставочный переулок,17,ЦАО,Пресненский район
1,улица Гашека,17,ЦАО,Пресненский район
2,Большая Никитская улица,17,ЦАО,Пресненский район
3,Глубокий переулок,17,ЦАО,Пресненский район
4,Большой Гнездниковский переулок,17,ЦАО,Пресненский район


Получим общую информацию о таблицах. Начнем с общих данных о заведениях. 

In [8]:
rest_data.info() # получение общей информации о данных в таблице rest_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


Таблица `rest_data` (данные о заведениях общественного питания)

В таблице 6 столбцов. Согласно документации к данным:
* `id` — идентификатор объекта, тип данных int64;
* `number` —количество посадочных мест, тип данных int64;

У остальных столбцов тип данных - object: 
* `object_name` — название объекта общественного питания;
* `chain` — сетевой ресторан;
* `object_type` — тип объекта общественного питания;
* `address` -адрес.

В таблице 15 366 значений в каждом столбце, скорее всего пропущенных значений нет.

Переименуем столбец `number` в `total_seats`, что лучше соответствует характеру данных.
Заменим формат столбца `chain` на boolean.

In [9]:
streets.info() # получение общей информации о данных в таблице streets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   streetname  4398 non-null   object
 1   areaid      4398 non-null   int64 
 2   okrug       4398 non-null   object
 3   area        4398 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.6+ KB


Таблица `streets` (информация о улицах)

В таблице 4 столбца. Документации к данным нет, но анализ содержимого таблицы показал, таблица содержит следующие столбцы:

* `streetname` — название улицы, тип данных object;
* `areaid` — идентификатор района, в котором расположена улица, тип данных int64;
* `okrug` — название округа, в котором расположена улица, тип данных object;
* `area` — название района, в котором расположена улица, тип данных object.

В таблице 4 398 значений в каждом столбце, скорее всего пропущенных значений нет.
Здесь необходимо привести названия столбцов `streetname`, `areaid` к хорошему стилю, а столбец `okrug` - перевеименовать в `district` для едиообразия.

Проверим, есть ли в таблице улицы, принадлежащие сразу нескольким районам. Посмотрим пример такой улицы.

In [10]:
# Подсчет названий улиц, встречающихся более 1 раза 
print('Всего дубликатов в названиях улиц', streets['streetname'].duplicated().sum())
# Демонстрация примера такого названияя
display(streets.loc[streets['streetname'].duplicated()].sort_values(by='streetname').head(1))

Всего дубликатов в названиях улиц 794


,streetname,areaid,okrug,area
4213,1-й Басманный переулок,15,ЦАО,Красносельский район


Проверим, в каких районах есть 1-й Басманный переулок.

In [11]:
# Получение информации о улице, встречающейся более одного раза
streets.query("streetname == '1-й Басманный переулок'")

,streetname,areaid,okrug,area
805,1-й Басманный переулок,13,ЦАО,Басманный район
4213,1-й Басманный переулок,15,ЦАО,Красносельский район


**Выводы**

Данные разделены на 2 таблицы. В первой содержатся описание и характеристики заведений общественного питания Москвы. 
Вторая таблица содержит информацию о связи улицы, района и административного округа. Для достижения целей исследования необходимо будет провести предобработку: привести данные к необходимым форматам, привести названия столбцов к хорошему стилю и убедиться в качестве данных. 

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Предобработка данных

Обработаем названия столбцов, исключим пропуски данных и дубликаты, проверим данные на наличие аномалий и при необходиЫмости избавимся от них. Приведем данные к необходимым типам.

#### Приведение названий столбцов к хорошему стилю

Заменим названия столбцов в таблице rest_data - приведем к нижнему регистру.

In [12]:
# Переименование столбцов
rest_data.rename(columns = {'number':'total_seats'}, inplace = True)

# Проверка - вывод названий столбцов
rest_data.columns 

Index(['id', 'object_name', 'chain', 'object_type', 'address', 'total_seats'], dtype='object')

Изменим стиль названий в таблице orders

In [13]:
# Переименование столбца
streets.rename(columns = {'streetname':'street_name', 'areaid' : 'area_id', 'okrug' : 'district'}, inplace = True)
# Проверка - вывод названий столбцов
streets.columns 

Index(['street_name', 'area_id', 'district', 'area'], dtype='object')

Теперь названия столбцов в порядке. 

#### Преобразование типов данных

Преобразуем данные в нужные типы. В столбцe `chain` необходимо заменить формат object на boolean.

Начнем с преобразований значений.

In [14]:
# Замена значений столбца на True и False
rest_data['chain'] = rest_data['chain'].replace({'да': True, 'нет': False})
# Запрос получившегося типа данных
print(type(rest_data['chain'].dtype))
# Проверка - получение первых 5 строк таблицы
rest_data.head()

<class 'numpy.dtype[bool_]'>


,id,object_name,chain,object_type,address,total_seats
0,151635,СМЕТАНА,False,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,False,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,False,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,True,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,False,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50


Замена произведена удачно. Теперь проверим данные на аномалии и дубликаты.

#### Проверка данных на аномалии и дубликаты

Определим, есть ли в таблицах rest_data и streets пропуски.

In [15]:
# Запрос количества пропущенных значений по столбцам.
print(rest_data.isna().sum())
print(streets.isna().sum())

id             0
object_name    0
chain          0
object_type    0
address        0
total_seats    0
dtype: int64
street_name    0
area_id        0
district       0
area           0
dtype: int64


В таблицах, как мы и предположили, отсутствуют пропуски.

Проверим поле total_seats на аномалии.

In [16]:
# Запрос характеристик столбца
rest_data['total_seats'].describe()

count    15366.000000
mean        59.547182
std         74.736833
min          0.000000
25%         12.000000
50%         40.000000
75%         80.000000
max       1700.000000
Name: total_seats, dtype: float64

Итак, мы видим, что в наших данных есть информация о разных заведениях - у некоторых совсем нет посадочных мест, а у других их очень много. Убедимся, что заведения с наибольшим количеством мест - не техническая ошибка, а реально работающие заведения.

In [17]:
rest_data.sort_values(by='total_seats', ascending=False).head(10)

,id,object_name,chain,object_type,address,total_seats
12723,199696,Arena by Soho Family,False,бар,"город Москва, Кутузовский проспект, дом 36, ст...",1700
8148,80291,Банкетный зал Шелк,False,ресторан,"город Москва, Большой Саввинский переулок, дом...",1500
8345,27750,СТОЛОВАЯ-БУФЕТ,False,столовая,"город Москва, улица Волхонка, дом 15",1400
9064,19719,КОМБИНАТ ПИТАНИЯ «УПРАВЛЕНИЕ АКАДЕМСЕРВИС»,False,столовая,"город Москва, проспект Вернадского, дом 84, ст...",1288
4480,27024,РУМ СЕРВИС,False,ресторан,"город Москва, Кутузовский проспект, дом 2/1, с...",1200
3686,27026,РУМ СЕРВИС,False,ресторан,"город Москва, площадь Европы, дом 2",1200
2313,26560,Ресторан «АЛЬФА»,False,ресторан,"город Москва, Измайловское шоссе, дом 71, корп...",1040
9955,171116,EATALY,False,ресторан,"город Москва, Киевская улица, дом 2",920
5649,29232,СТОЛОВАЯ МОСКОВСКИЙ ГУМАНИТАРНЫЙ УНИВЕРСИТЕТ,False,столовая,"город Москва, улица Юности, дом 5, строение 1",760
13299,200905,White Hall,False,ресторан,"город Москва, Ферганская улица, дом 8",700


Итак, аномально высокие показатели количества посадочных мест объясняются тем, что это совокупные показатели нескольких заведений - все столовые вуза (например, РАНХИГС - «УПРАВЛЕНИЕ АКАДЕМСЕРВИС» или МГУ - СТОЛОВАЯ МОСКОВСКИЙ ГУМАНИТАРНЫЙ УНИВЕРСИТЕТ), все точки питания большой гостиницы (как .Ресторан «АЛЬФА» в гостинице Измайлово). Чтобы эти данные не влияли на наши результаты, необходимо удалить их. В качестве границы определим 500 мест - рестораны такого размера уже могут обслуживать посетителей.

In [18]:
# Удаление из таблицы заведений с числом посадочных мест более 500.
rest_data = rest_data.loc[rest_data['total_seats'] < 500]
# Проверка - получение количесва строк таблицы
print('Количество записей в таблице данных заведений общественного питания', len(rest_data))

Количество записей в таблице данных заведений общественного питания 15332


Теперь проверим таблицу rest_data на скрытые дубликаты. Сначала проанализируем виды заведений, а потом обработаем их названия.

In [19]:
# Запрос значений столбца
rest_data['object_type'].value_counts()

кафе                                 6098
столовая                             2577
ресторан                             2264
предприятие быстрого обслуживания    1922
бар                                   855
буфет                                 585
кафетерий                             398
закусочная                            360
магазин (отдел кулинарии)             273
Name: object_type, dtype: int64

Мы будем использовать объекты для подписей в графиках, поэтому следует адаптировать тип "предприятие быстрого обслуживания для подписей".

In [20]:
rest_data.loc[rest_data['object_type'] == 'предприятие быстрого обслуживания','object_type'] = 'фаст-фуд'

Дубликатов в столбце с видами заведений нет, однако мы можем удалить из анализа закусочные и магазин (отдел кулинарии), так как это специфические виды заведений, информация о них не обогатит исследование для нашего кафе.

In [21]:
# Удаление из таблицы заведений с типом "закусочная" или "магазин (отдел кулинарии)".
rest_data = rest_data.loc[~rest_data['object_type'].isin(['закусочная', 'магазин (отдел кулинарии)'])]
# Проверка - получение количесва строк таблицы
print('Количество записей в таблице данных заведений общественного питания', len(rest_data))

Количество записей в таблице данных заведений общественного питания 14699


Теперь проанализируем названия заведений.

In [22]:
# Запрос уникальных значений столбца
sorted(rest_data['object_name'].unique())
# Проверка - запрос количества уникальных названий
print('Количество оригинальных названий - ', rest_data['object_name'].nunique())

Количество оригинальных названий -  10116


Мы видим, что в столбце есть неявные дубликаты + иногда в название попадает тип заведения (например, кафе "Роза"). В нашу задачу не входит устранение проблем в этом поле, однако мы можем привести все данные к единому регистру и убрать из названий тип.  

In [23]:
# Преобразование регистра
rest_data['object_name'] = rest_data['object_name'].str.title()
# Проверка - запрос количества уникальных названий
print('Количество оригинальных названий - ', rest_data['object_name'].nunique() )

Количество оригинальных названий -  9676


In [24]:
# Удаление вида заведения
rest_data['object_name'] = rest_data['object_name'].str.replace('Кафе', '').str.strip()
rest_data['object_name'] = rest_data['object_name'].str.replace('Ресторан', '').str.strip()
rest_data['object_name'] = rest_data['object_name'].str.replace('Банкетный Зал', '').str.strip()
rest_data['object_name'] = rest_data['object_name'].str.replace('Бар', '').str.strip()
# Проверка - запрос количества уникальных названий
print('Количество оригинальных названий - ', rest_data['object_name'].nunique() )

Количество оригинальных названий -  9471


Теперь проверим таблицы на явные дубликаты.

In [25]:
# Запрос количества явных дубликатов
print(rest_data.duplicated().sum())
print(streets.duplicated().sum())

0
8


Удалим явные дубликаты из таблицы улиц.

In [26]:
# Удаление явных дубликатов
streets = streets.drop_duplicates().reset_index(drop=True) 
streets.duplicated().sum() # проверка на отсутствие дубликатов

0

Явных дубликатов в таблицах нет. 
Мы проверили данные на аномалии и дубликаты, устранили небольшие погрешности и теперь можем приступать к дальнейшему анализу.

## Анализ данных

В рамках анализа данных проанализируем соотношение видов объектов общественного питания, рассмотрим особенности сетевых и несетевых заведений, выделим информацию об улицах, на которых расположены заведения и выявим улицы с высоким количеством заведения и с одним заведением на всей улице, определим районы, где эти улицы расположены.

### Виды объектов общественного питания: какие более распространены?

Исследуем соотношение видов объектов общественного питания по количеству.

Зададим стиль для графиков seaborn.

In [27]:
# Определение размера элементов и шрифта
sns.set_context("paper", font_scale = 2)
# Определение стиля графика
sns.set_style('ticks')
# Определение палитры 
sns.set_palette("rainbow")

In [28]:
# Определение размера графика
plt.figure(
    figsize=(7, 9)
)  
# Построение диаграммы с распределением количества по категориям
ax = sns.countplot(
    y='object_type', 
    data=rest_data,
    order=rest_data['object_type'].value_counts().index, 
    color='#1F77B4',
)
ax.set(xlim=(0, 7000))
sns.despine()
# Добавление подписей столбцов
ax.bar_label(ax.containers[0], label_type='edge', fontsize=16)
# Добавление подписей графика и осей
plt.title('Количество объектов питания в городе Москва: \nраспределение по видам объектов, шт.')
plt.xlabel('Количество, шт.')
plt.ylabel('Вид объекта питания')
# Демонстрация графика
plt.show()
# 
plt.savefig('dkdkd.png')

TypeError: got an unexpected keyword argument 'orientation'

<Figure size 504x648 with 1 Axes>

TypeError: got an unexpected keyword argument 'facecolor'

<Figure size 432x288 with 0 Axes>

In [ ]:
print('Всего объектов общественного питания в городе Москва', rest_data['object_type'].count())

**Вывод**

Среди видов объектов общественного питания более трети - кафе, на столовые приходится около 17% заведений, на рестораны - около 15%. На предприятия быстрого обслуживания приходится около 13% заведений, остальные виды распространены значительно меньше.

### Сетевое распространение: какие виды и характеристики у сетевых заведений?

Исследуем соотношение сетевых и несетевых заведений по количеству. 

In [ ]:
# Определение размера графика
plt.figure(
    figsize=(7, 9)
)  
# Построение диаграммы с распределением количества по категориям
ax = sns.countplot(
    y='chain', 
    data=rest_data,
    order=[True, False],
    palette=['#FF770E',"#1F77B4"]
)
ax.set_yticklabels(['Сетевое\nзаведение', 'Несетевое\nзаведение'])
ax.set(xlim=(0, 13000))
# Добавление подписей столбцов
ax.bar_label(ax.containers[0], label_type='edge', fontsize=16)
sns.despine() 
# Добавление подписей графика и осей
plt.title('Количество объектов питания в городе Москва: \nраспределение по принадлежности к сети, шт.')
plt.xlabel('Количество, шт.')
plt.ylabel('Принадлежность к сети')
# Демонстрация графика
plt.show()

К сетевым заведениям относится около 1/5 (19.3%) всех заведений общественного питания. Эти доли могут быть неоднородно распределены по видам заведений. Проверим, для каких видов объектов общественного питания характерно сетевое распространение. Построим сводную таблицу с информацией о количестве заведений и мест в них с сетевым и несетевым распространением. В качестве показателя типичного заведения выберем медианное количество мест, так как этот показатель более устойчив к выбросам.

In [ ]:
# Построение сводной таблицы с данными об объектах общественного питания
rest_stats = rest_data.pivot_table(
    index = ['object_type'], 
    values = 'total_seats', 
    columns = 'chain',
    aggfunc=['count', 'sum', 'median'],
)
# Переименование столбцов
rest_stats.columns = (['not_chain_objects', 'chain_objects', 
                       'not_chain_total_seats', 'chain_total_seats', 
                       'non_chain_median_seats', 'chain_median_seats']
                     )
# Округление данных о медианном количестве мест
rest_stats['non_chain_median_seats'] = round(rest_stats['non_chain_median_seats'])
rest_stats['chain_median_seats'] = round(rest_stats['chain_median_seats'])

# Расчет доли объектов сетевого распространения
rest_stats['chain_object_share'] = (round(rest_stats['chain_objects'] /
                                           (rest_stats['chain_objects'] + 
                                            rest_stats['not_chain_objects']), 
                                           2)
                                    )
# Расчет доли мест в объектах сетевого распространения
rest_stats['chain_seats_share'] = (round(rest_stats['chain_total_seats'] /
                                           (rest_stats['chain_total_seats'] + 
                                            rest_stats['not_chain_total_seats']), 
                                           2)
                                    ) 
# Сортировка по количеству объектов сетевого распространения
rest_stats = rest_stats.sort_values(by='chain_objects')
# Переименование столбцов для графиков
rest_stats_for_graphics = rest_stats.rename({'not_chain_objects':'несетевые заведения', 'chain_objects':'сетевые заведения'}, axis=1)
# Демонстрация таблицы
display(rest_stats)

По полученным данным построим диаграмму.

In [ ]:
# Построение линейчатой диаграммы
fig = px.bar(rest_stats_for_graphics, 
             y=rest_stats_for_graphics.index, 
             x=['несетевые заведения', 'сетевые заведения'],
# Добавление названия таблицы, осей и легенды             
             labels={
                     "object_type": "Вид  объекта",
                     'value': 'Количество, шт.', 
                     'variable': 'Вид объекта'
                 },
             title="Распределение сетевых и несетевых заведений по видам предприятия общественного питания ",            
# Выбор шаблона             
             template='simple_white',
            )
# Демонстрация диаграммы
fig.show()

Сетевое распространение в относительном отношении в первую очередь характерно для предприятий быстрого обслуживания, на сетевые заведения здесь приходится 41% объектов и 80% посадочных мест.
Также значительная часть объектов кафе - 23% и ресторанов 24% относятся к сетевому типу организации бизнеса. Учитывая долю кафе на рынке, в абсолютном отношении количество сетевых кафе больше, чем количество предприятий быстрого питания.

Что же характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест? Проанализируем эти заведения в сравнении с несетевыми.
Подготовим данные для постройки диаграмм.

In [ ]:
# Выбор данных для графиков: принадлежность к сети и наиболее популярный формат: кафе, ресторан, фаст-фуд
rd_chain_cafe = rest_data.loc[(rest_data['chain']==True) & (rest_data['object_type']=='кафе')]
rd_non_chain_cafe = rest_data.loc[(rest_data['chain']==False) & (rest_data['object_type']=='кафе')]
rd_chain_rest = rest_data.loc[(rest_data['chain']==True) & (rest_data['object_type']=='ресторан')]
rd_non_chain_rest = rest_data.loc[(rest_data['chain']==False) & (rest_data['object_type']=='ресторан')]
rd_chain_fast_food = rest_data.loc[(rest_data['chain']==True) & (rest_data['object_type']=='фаст-фуд')]
rd_non_chain_fast_food = rest_data.loc[(rest_data['chain']==False) & (rest_data['object_type']=='фаст-фуд')]

Сравним сетевые и несетевые заведения по наиболее популярным форматам: кафе, ресторан, фаст-фуд. Построим диаграмму размаха для этих видов объектов общественного питания

In [ ]:
# Создание графического объекта
fig = go.Figure()
# Добавление диаграммы размаха по каждому виду объектов, определение названия и цвета
fig.add_trace(go.Box(y=rd_chain_cafe['total_seats'], name='Сеть: кафе',
                marker_color = '#FF7F0E'))
fig.add_trace(go.Box(y=rd_chain_rest['total_seats'], name='Сеть: ресторан',
                marker_color = '#FF7F0E'))
fig.add_trace(go.Box(y=rd_chain_fast_food['total_seats'], name='Сеть: фаст-фуд',
                marker_color = '#FF7F0E'))
fig.add_trace(go.Box(y=rd_non_chain_cafe['total_seats'], name = 'Не сеть: кафе',
                marker_color = '#1F77B4'))
fig.add_trace(go.Box(y=rd_non_chain_rest['total_seats'], name = 'Не сеть: ресторан',
                marker_color = '#1F77B4'))
fig.add_trace(go.Box(y=rd_non_chain_fast_food['total_seats'], name = 'Не сеть: фаст-фуд',
                marker_color = '#1F77B4'))
# Обновление шаблона: добавление названия диаграммы и осей, шаблона и удаление легенды
fig.update_layout(
    title="Распределение сетевых и несетевых заведений по количеству мест",
    xaxis_title="Принадлежность к сети и вид объекта",
    yaxis_title="Количество мест",
    template='simple_white',
    showlegend=False
)
# Демонстрация диаграммы
fig.show()

In [ ]:
fig.write_image("fig2.png")

Внутри сетевых заведений есть существенные отличия, в зависимости от вида, однако общая тенденция такова: сетевых заведений меньше и они имеют больше посадочных мест, чем несетевые заведения. Так для количества посадочных мест медиана сетевый предприятий быстрого обслуживания составляет 25 мест, у несетевых - 0. У кафе соотношение медиан 40 к 30 в пользу сети, у ресторанов - 88 к 75 в пользу сети.

**Вывод**
Сетевое распространение наиболее характерно для предприятий быстрого питания, достаточно распространено среди ресторанов и кафе. Для этого типа заведений харакерно меньшее число объектов с большим количеством посадочных мест.

### Количество мест в заведении: какие виды объектов лидируют?

Подробнее рассмотрим общую информацию о среднем количестве посадочных мест в заведениях разного вида. 

Для каждого вида объекта общественного питания опишем среднее количество посадочных мест. 

In [ ]:
# Создание сводной таблицы
seats_stats = rest_data.pivot_table(
    index = ['object_type'], 
    values = 'total_seats',
    aggfunc=['count', 'median', 'mean'],
)
seats_stats.columns = (['object_count', 'total_seats_median', 'total_seats_mean'])
# Округление данных о медианном и среднем количестве мест
seats_stats['total_seats_median'] = round(seats_stats['total_seats_median'])
seats_stats['total_seats_mean'] = round(seats_stats['total_seats_mean'])
# Сортировка по среднему количеству мест
seats_stats = seats_stats.sort_values(by='total_seats_mean', ascending=False)
# Демонстрация таблицы
display(seats_stats)

По полученным данным пострроим диаграмму.

In [ ]:
seats_stats.index

In [ ]:
# Определение размера графика
plt.figure(
    figsize=(7, 9)
)  
# Построение диаграммы с распределением количества по категориям
ax = sns.barplot(
    y='object_type', 
    x='total_seats',
    data=rest_data,
    order=seats_stats.index, 
    color='#1F77B4',
    ci=None,
)
ax.set(xlim=(0, 150))
sns.despine() 
# Добавление подписей столбцов
ax.bar_label(ax.containers[0], label_type='edge', fontsize=16)
# Добавление подписей графика и осей
plt.title('Среднее количество мест в объекте питания в городе Москва:\nраспределение по виду объекта, чел.')
plt.xlabel('Количество, шт.')
plt.ylabel('Вид объекта питания')
# Демонстрация графика
plt.show()

**Вывод**

В среднем самое большое количество посадочных мест предоставляют столовые - почти 128. Достаточно много мест в ресторанах - около 91. Кафе, бары и буфеты предлагают 40-50 мест в среднем, фаст-фуд - около 20, а кафетерий - меньше 9.

###  Расположение заведения: влияние улицы и района

Выделим в отдельный столбец информацию об улице из столбца address . Для этого воспользуемся регулярными выражениями и срезами данных.

In [ ]:
# Функция возвращает строку без первых 14 символов
def remove_start(address, symbol=14):
    return address[symbol:]
# Функция возвращает всю строку до указателя дома
def remove_end(address, sep=', дом',):
    return address.split(sep)[0]

# Преобразования адреса в улицу
rest_data['street_name'] = rest_data['address'].apply(lambda x: remove_start(x))
rest_data['street_name'] = rest_data['street_name'].apply(lambda x: remove_end(x))
rest_data['street_name'] = rest_data['street_name'].apply(lambda x: remove_end(x, ', владение'))

print('Всего уникальных названий улиц в таблице -', rest_data['street_name'].nunique())
# Проверка - вывод первых 5 строк таблицы.
rest_data.head()

Построим график топ-10 улиц по количеству объектов общественного питания. 

In [ ]:
rest_data['street_name'].value_counts().head(10)

In [ ]:
# Определение размера графика
plt.figure(
    figsize=(7, 9)
)  
# Построение диаграммы с распределением количества по категориям
ax = sns.barplot(
    y=rest_data['street_name'].value_counts().head(10).index, 
    x=rest_data['street_name'].value_counts().head(10),
    color='#1F77B4',
    ci=None,
)
ax.set(xlim=(0, 200))
sns.despine() 
# Добавление подписей столбцов
ax.bar_label(ax.containers[0], label_type='edge', fontsize=16)
# Добавление подписей графика и осей
plt.title('ТОП-10 улиц по количеству заведений общественного питания,\n расположенных на них')
plt.xlabel('Количество заведений, шт.')
plt.ylabel('Улица')
# Демонстрация графика
plt.show()

На каждой из наиболее популярных улиц расположено более 100 заведений общественного питания. Улица, возглавляющая рейтинг - проспект Мира. Благодаря тому, что он имеет большую протяженность и на нем расположено сразу несколько крупных торговых центров, число предприятий питания там достигает 192.

Воспользуйтесь внешней информацией и ответьте на вопрос — в каких районах Москвы находятся эти улицы? 3

Определим, в каких районах расположены улицы - лидеры.

In [ ]:
# Объединение таблиц с информацией об улице и районе
top_streets = streets[['street_name', 'district', 'area']].join(
    rest_data['street_name'].value_counts().head(10), 
                           on='street_name', 
                           lsuffix='', 
                           rsuffix='1', 
                           how='right'
                          )
# 
top_streets = top_streets.drop(columns=['street_name1'])
display(top_streets)

Определим, есть ли районы или округа, в которых проходят сразу нескоолько улиц-лидеров.

In [ ]:
# Демонстрация таблицы
display(top_streets['area'].value_counts())

В 11 районах есть по 2 улицы-лидера.

In [ ]:
display(top_streets['district'].value_counts())

Из округов наиболее популярные - Южный и Юго-Западный - в них по 14 комбинаций районов-улиц с максимальным количеством предприятий питания.

Найдем число улиц с одним объектом общественного питания. 

In [ ]:
len(rest_data['street_name'].value_counts().loc[rest_data['street_name'].value_counts() == 1])

Всего улиц, на которых расположен лишь один объект общественного питания в нашем массиве 754. Определим, в каких районах Москвы находятся эти улицы.

In [ ]:
# Объединение таблиц с информацией об улице и районе
rare_streets = streets[['street_name', 'district', 'area']].join(
    rest_data['street_name'].value_counts().loc[rest_data['street_name'].value_counts() == 1], 
                           on='street_name', 
                           lsuffix='', 
                           rsuffix='1', 
                           how='inner'
                          )
# 
rare_streets = rare_streets.drop(columns=['street_name1'])
display(rare_streets.sort_values(by='street_name').head(10))

Определим, в каких районах и административных округах находятся менее популярные улицы.

In [ ]:
# Демонстрация таблиц
display(rare_streets['area'].value_counts().head(20))

Многие районы центрального административного округа имеют улицы с одним заведением. Это может быть связано с особенностью района - исторический центр и короткие улицы. 

In [ ]:
display(rare_streets['district'].value_counts())

Среди районов с улицами, на которых находится одно заведение предсказуемо лидирует Центральный - по причинам описанным выше. Также достаточно часто встречаются такие улицы в ВАО, СВАО, ЮВАО, ЗАО.

**Вывод**

Топографически объекты общественного питания размещены неравномерно. На самых популярных улицах работают свыше сотни объектов питания, а более 700 улиц в нашей базе данных представлены лишь одним заведением. Наиболее популярная улица - проспект Мира, административные округа с популярными улицами - ЮЗАО, ЮАО, с наименее популярными улицами - ЦАО, ВАО, СВАО, САО, ЮВАО.

## Подготовка презентации

Презентация: <https://disk.yandex.ru/i/rCmscX2nkbx_Qw> .

## Общий вывод

Таким образом, относительно развития кафе с роботизированным обслуживанием в Москве можно дать следующие рекомендации.

1. Учитывая концепцию дорогого и небольшого заведения, перспективным видом предприятия питания можно признать кафе: рестораны имеют более высокое число посадочных мест, а фаст-фуд - предполагает сетевую организацию бизнеса.
2. Предлагаемое количество посадочных мест - 30. Это соответствует медианнному значению для несетевых кафе.
3. Приоритетный район расположения - Юго-Западное направение, ЮАО, ЮЗАО, конкретные районы - Чертаново, Ломоносовский, Нагаткино-Садовники и другие, включая улицы Профсоюзная и Кировоградская, Варшавское и Каширское шоссе, Ленинский проспект.
4. В условиях ориентации на небольшое и дорогое место концепция сети пока представляется нецелесообразной, однако при успешном запуске первого кафе и оптимизации расходов можно будет подумать об открытии сети, при условии: - повышении количества посадочных мест, - ориентации на "проходимые" и активные районы, прежде всего на Юге и Юго-Западе столицы.